In [401]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


11/7 評価関数F2を各手法ごとに追記(上書きでミスってた時が怖いため，ファイル名を"データセット-手法-addF2.csv"に変更)

In [448]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
#sysはコマンドライン引数
import sys
import csv
import math
c = 3

In [449]:
#fcm,fecbo-dbクラスタ中心計算部分関数作成
def v_calculation(u_m,c,dimension,X):
    #分母計算(クラスタサイズになるはず)
    v_denominator = np.sum(u_m,axis=0)
    #分子計算
    v_numerator = np.zeros((c,dimension))
    for j in range(c):
        for i in range(X_size):
            v_numerator[j]+= u_m[i][j]*X[i]
   
    v = np.zeros((c,dimension ))
    for i in range(c):
        v[i] = v_numerator[i]/v_denominator[i]
    return v

In [459]:
#引数(クラスタ数(c),クラスター(cluster),データセット(X),クラスター中心(v)、\
#ファジィ化パラム(param(mかlam_1))、帰属度(u))

def calc_F(c,cluster,X,v,param,u,model):
    #中心間距離の作成
    center_distance = np.zeros((c,c))
    for i in range(c):
        for j in range(c):
            center_distance[i][j]= np.sum((v[i]-v[j])**2)
    print("center_distance")
    print(center_distance)
    #dにはd11,d12,d13,d14,d21,d22,,,dn1,,dncの順でリスト作る
    d_list=[]
    for i in range(X_size):
        for j in range(c):
            d_list.append(np.sum((X[i]-v[j])**2))
    d = np.array(d_list)
    d = d.reshape([X_size,c])
    print("d[0]",d[0])

    new_d = np.zeros((X_size,c))
    #ここが怪しい
    for i in range(c):
        new_d[cluster == i] = d[cluster == i]+center_distance[i]
    print("new_d[0] = d_{ki}+d_{ii*}",new_d[0])

    #評価関数F_1計算(ついでにJも再計算してあってるか確認)
    F_1 = 0
    for i in range(X_size):
        F_1 += np.sum(u[i] * new_d[i])
    #目的関数の再計算（確認）
    kakunin_J = 0
    if (model =="fecbo-db") or (model == "fcm"):
        u_m = u**param
        for i in range(X_size):
            kakunin_J += np.sum(u_m[i] * d[i])
    if model =="fecbo-e":
        for i in range(X_size):
            for j in range(c):
                kakunin_J += u[i][j]*d[i][j]+param*u[i][j]*math.log(u[i][j])
    
        
    #評価関数F_2計算
    F_2 = 0
    new_u = np.copy(u)
    for i in range(X_size):
        new_u[i][cluster[i]] = 0
    for i in range(X_size):
        F_2 += np.sum(new_u[i] * new_d[i])
    #この初期値のときの結果が優れている場合，bestを更新
    print("Jの値:",J)
    print("Jの値を再計算:",kakunin_J)
    print("F_1の値:",F_1)
    print("F_2の値:",F_2)
    return F_1,F_2

## FCMへのF2の記入

In [405]:
df = pd.read_csv("./iris_csvFile/Iris-fcm.csv")
u = df.loc[:,"u1":"u3"].values
X = df.loc[:,"sepal_length":"petal_width"].values
F = df["F"].values[0]
J = df["J"].values[0]
cluster = df["cluster"].values
m = 3
u_m = u**m
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F,F2
0,5.1,3.5,1.4,0.2,0,0.915676,0.034733,0.049591,29.110238,834.025086,786.268279
1,4.9,3.0,1.4,0.2,0,0.827251,0.070570,0.102178,29.110238,834.025086,786.268279
2,4.7,3.2,1.3,0.2,0,0.835861,0.067695,0.096444,29.110238,834.025086,786.268279
3,4.6,3.1,1.5,0.2,0,0.801531,0.081068,0.117401,29.110238,834.025086,786.268279
4,5.0,3.6,1.4,0.2,0,0.894932,0.043400,0.061669,29.110238,834.025086,786.268279
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1,0.069029,0.715875,0.215096,29.110238,834.025086,786.268279
146,6.3,2.5,5.0,1.9,1,0.097676,0.466216,0.436108,29.110238,834.025086,786.268279
147,6.5,3.0,5.2,2.0,1,0.064641,0.698666,0.236693,29.110238,834.025086,786.268279
148,6.2,3.4,5.4,2.3,1,0.093683,0.624896,0.281420,29.110238,834.025086,786.268279


In [406]:
v = v_calculation(u_m,c,dimension,X)
v

array([[5.00106653, 3.38935647, 1.49425962, 0.25194824],
       [6.69509539, 3.03751182, 5.55144409, 2.03544793],
       [5.90997311, 2.79144758, 4.37839845, 1.39638155]])

In [407]:
F_1,F_2 = calc_F(c,cluster,X,v,m,u)

center_distance
[[ 0.         22.63514541 10.81159058]
 [22.63514541  0.          2.46140653]
 [10.81159058  2.46140653  0.        ]]
d[0] [ 0.03361332 23.36158179 11.46028911]
new_d[0] = d_{ki}+d_{ii*} [3.36133186e-02 4.59967272e+01 2.22718797e+01]
Jの値: 29.11023838969788
Jの値を再計算: 29.11023838969677
F_1の値: 834.0250861732662
F_2の値: 786.2682785314418


In [408]:
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F,F2
0,5.1,3.5,1.4,0.2,0,0.915676,0.034733,0.049591,29.110238,834.025086,786.268279
1,4.9,3.0,1.4,0.2,0,0.827251,0.070570,0.102178,29.110238,834.025086,786.268279
2,4.7,3.2,1.3,0.2,0,0.835861,0.067695,0.096444,29.110238,834.025086,786.268279
3,4.6,3.1,1.5,0.2,0,0.801531,0.081068,0.117401,29.110238,834.025086,786.268279
4,5.0,3.6,1.4,0.2,0,0.894932,0.043400,0.061669,29.110238,834.025086,786.268279
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1,0.069029,0.715875,0.215096,29.110238,834.025086,786.268279
146,6.3,2.5,5.0,1.9,1,0.097676,0.466216,0.436108,29.110238,834.025086,786.268279
147,6.5,3.0,5.2,2.0,1,0.064641,0.698666,0.236693,29.110238,834.025086,786.268279
148,6.2,3.4,5.4,2.3,1,0.093683,0.624896,0.281420,29.110238,834.025086,786.268279


In [409]:
df["F2"] = F_2
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F,F2
0,5.1,3.5,1.4,0.2,0,0.915676,0.034733,0.049591,29.110238,834.025086,786.268279
1,4.9,3.0,1.4,0.2,0,0.827251,0.070570,0.102178,29.110238,834.025086,786.268279
2,4.7,3.2,1.3,0.2,0,0.835861,0.067695,0.096444,29.110238,834.025086,786.268279
3,4.6,3.1,1.5,0.2,0,0.801531,0.081068,0.117401,29.110238,834.025086,786.268279
4,5.0,3.6,1.4,0.2,0,0.894932,0.043400,0.061669,29.110238,834.025086,786.268279
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1,0.069029,0.715875,0.215096,29.110238,834.025086,786.268279
146,6.3,2.5,5.0,1.9,1,0.097676,0.466216,0.436108,29.110238,834.025086,786.268279
147,6.5,3.0,5.2,2.0,1,0.064641,0.698666,0.236693,29.110238,834.025086,786.268279
148,6.2,3.4,5.4,2.3,1,0.093683,0.624896,0.281420,29.110238,834.025086,786.268279


In [286]:
df.to_csv('./iris_csvFile/Iris-fcm-test.csv',index = False)
# df.to_csv('./iris_csvFile/Iris-fcm.csv',index = False)

In [283]:
# df.drop("F_2",axis=1,inplace=True)
# df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F
0,5.1,3.5,1.4,0.2,0,0.915676,0.034733,0.049591,29.110238,834.025086
1,4.9,3.0,1.4,0.2,0,0.827251,0.070570,0.102178,29.110238,834.025086
2,4.7,3.2,1.3,0.2,0,0.835861,0.067695,0.096444,29.110238,834.025086
3,4.6,3.1,1.5,0.2,0,0.801531,0.081068,0.117401,29.110238,834.025086
4,5.0,3.6,1.4,0.2,0,0.894932,0.043400,0.061669,29.110238,834.025086
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1,0.069029,0.715875,0.215096,29.110238,834.025086
146,6.3,2.5,5.0,1.9,1,0.097676,0.466216,0.436108,29.110238,834.025086
147,6.5,3.0,5.2,2.0,1,0.064641,0.698666,0.236693,29.110238,834.025086
148,6.2,3.4,5.4,2.3,1,0.093683,0.624896,0.281420,29.110238,834.025086


## FECBO-DB(M=3)へのF2の記入

In [410]:
df = pd.read_csv("./iris_csvFile/Iris-m3.csv")
u = df.loc[:,"u0":"u2"].values
X = df.loc[:,"sepal_length":"petal_width"].values
F = df["F"].values[0]
J = df["J"].values[0]
cluster = df["cluster"].values
m = 3
u_m = u**m
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2,J,F
0,5.1,3.5,1.4,0.2,1.0,0.060611,0.885228,0.054161,29.289736,835.958407
1,4.9,3.0,1.4,0.2,1.0,0.108113,0.810330,0.081557,29.289736,835.958407
2,4.7,3.2,1.3,0.2,1.0,0.101756,0.820053,0.078190,29.289736,835.958407
3,4.6,3.1,1.5,0.2,1.0,0.122551,0.786852,0.090597,29.289736,835.958407
4,5.0,3.6,1.4,0.2,1.0,0.070286,0.870102,0.059612,29.289736,835.958407
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0,0.181321,0.054109,0.764570,29.289736,835.958407
146,6.3,2.5,5.0,1.9,2.0,0.396489,0.088777,0.514734,29.289736,835.958407
147,6.5,3.0,5.2,2.0,2.0,0.173925,0.041181,0.784894,29.289736,835.958407
148,6.2,3.4,5.4,2.3,2.0,0.257683,0.084530,0.657787,29.289736,835.958407


In [411]:
v = v_calculation(u_m,c,dimension,X)
v

array([[5.8798245 , 2.7805334 , 4.33476775, 1.37513018],
       [4.99946148, 3.39023264, 1.49106561, 0.25094682],
       [6.65877695, 3.0271593 , 5.50532537, 2.01772969]])

In [412]:
cluster = cluster.astype(np.int64)
cluster.dtype

dtype('int64')

In [413]:
F_1,F_2 = calc_F(c,cluster,X,v,m,u)

center_distance
[[ 0.         10.49720232  2.45073051]
 [10.49720232  0.         22.12095316]
 [ 2.45073051 22.12095316  0.        ]]
d[0] [11.11955116  0.03304539 22.8112015 ]
new_d[0] = d_{ki}+d_{ii*} [2.16167535e+01 3.30453901e-02 4.49321547e+01]
Jの値: 29.289735663504533
Jの値を再計算: 29.289735663503222
F_1の値: 835.9584066789752
F_2の値: 787.901128548432


In [414]:
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2,J,F
0,5.1,3.5,1.4,0.2,1.0,0.060611,0.885228,0.054161,29.289736,835.958407
1,4.9,3.0,1.4,0.2,1.0,0.108113,0.810330,0.081557,29.289736,835.958407
2,4.7,3.2,1.3,0.2,1.0,0.101756,0.820053,0.078190,29.289736,835.958407
3,4.6,3.1,1.5,0.2,1.0,0.122551,0.786852,0.090597,29.289736,835.958407
4,5.0,3.6,1.4,0.2,1.0,0.070286,0.870102,0.059612,29.289736,835.958407
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0,0.181321,0.054109,0.764570,29.289736,835.958407
146,6.3,2.5,5.0,1.9,2.0,0.396489,0.088777,0.514734,29.289736,835.958407
147,6.5,3.0,5.2,2.0,2.0,0.173925,0.041181,0.784894,29.289736,835.958407
148,6.2,3.4,5.4,2.3,2.0,0.257683,0.084530,0.657787,29.289736,835.958407


In [415]:
df["cluster"] = cluster
df["F2"] = F_2
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2,J,F,F2
0,5.1,3.5,1.4,0.2,1,0.060611,0.885228,0.054161,29.289736,835.958407,787.901129
1,4.9,3.0,1.4,0.2,1,0.108113,0.810330,0.081557,29.289736,835.958407,787.901129
2,4.7,3.2,1.3,0.2,1,0.101756,0.820053,0.078190,29.289736,835.958407,787.901129
3,4.6,3.1,1.5,0.2,1,0.122551,0.786852,0.090597,29.289736,835.958407,787.901129
4,5.0,3.6,1.4,0.2,1,0.070286,0.870102,0.059612,29.289736,835.958407,787.901129
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,0.181321,0.054109,0.764570,29.289736,835.958407,787.901129
146,6.3,2.5,5.0,1.9,2,0.396489,0.088777,0.514734,29.289736,835.958407,787.901129
147,6.5,3.0,5.2,2.0,2,0.173925,0.041181,0.784894,29.289736,835.958407,787.901129
148,6.2,3.4,5.4,2.3,2,0.257683,0.084530,0.657787,29.289736,835.958407,787.901129


In [416]:
df.to_csv('./iris_csvFile/Iris-m3-addF2.csv',index = False)
# df.to_csv('./iris_csvFile/Iris-fcm.csv',index = False)

## FECBO-DB(M=5)へのF2の記入

In [417]:
df = pd.read_csv("./iris_csvFile/Iris-m5.csv")
u = df.loc[:,"u0":"u2"].values
X = df.loc[:,"sepal_length":"petal_width"].values
F = df["F"].values[0]
J = df["J"].values[0]
cluster = df["cluster"].values
m = 5
u_m = u**m
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2,J,F
0,5.1,3.5,1.4,0.2,1.0,0.157119,0.694499,0.148381,4.25485,1454.160086
1,4.9,3.0,1.4,0.2,1.0,0.209519,0.605768,0.184713,4.25485,1454.160086
2,4.7,3.2,1.3,0.2,1.0,0.206081,0.611111,0.182808,4.25485,1454.160086
3,4.6,3.1,1.5,0.2,1.0,0.221548,0.584508,0.193945,4.25485,1454.160086
4,5.0,3.6,1.4,0.2,1.0,0.173359,0.667385,0.159256,4.25485,1454.160086
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0,0.280184,0.159885,0.559931,4.25485,1454.160086
146,6.3,2.5,5.0,1.9,2.0,0.375219,0.178483,0.446298,4.25485,1454.160086
147,6.5,3.0,5.2,2.0,2.0,0.240008,0.129790,0.630202,4.25485,1454.160086
148,6.2,3.4,5.4,2.3,2.0,0.316128,0.182531,0.501342,4.25485,1454.160086


In [418]:
v = v_calculation(u_m,c,dimension,X)
v

array([[5.93921272, 2.81808895, 4.39340728, 1.40452979],
       [5.00738141, 3.3845727 , 1.49985005, 0.25004123],
       [6.58090655, 3.0086593 , 5.40692902, 1.9725428 ]])

In [419]:
cluster = cluster.astype(np.int64)
cluster.dtype

dtype('int64')

In [420]:
F_1,F_2 = calc_F(c,cluster,X,v,m,u)

center_distance
[[ 0.         10.89473071  1.79795312]
 [10.89473071  0.         20.84956998]
 [ 1.79795312 20.84956998  0.        ]]
d[0] [11.58065984  0.03437582 21.63188804]
new_d[0] = d_{ki}+d_{ii*} [2.24753906e+01 3.43758222e-02 4.24814580e+01]
Jの値: 4.254850237086912
Jの値を再計算: 4.254850237086762
F_1の値: 1454.1600860571896
F_2の値: 1414.7922099217328


In [422]:
df["cluster"] = cluster
df["F2"] = F_2
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2,J,F,F2
0,5.1,3.5,1.4,0.2,1,0.157119,0.694499,0.148381,4.25485,1454.160086,1414.79221
1,4.9,3.0,1.4,0.2,1,0.209519,0.605768,0.184713,4.25485,1454.160086,1414.79221
2,4.7,3.2,1.3,0.2,1,0.206081,0.611111,0.182808,4.25485,1454.160086,1414.79221
3,4.6,3.1,1.5,0.2,1,0.221548,0.584508,0.193945,4.25485,1454.160086,1414.79221
4,5.0,3.6,1.4,0.2,1,0.173359,0.667385,0.159256,4.25485,1454.160086,1414.79221
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,0.280184,0.159885,0.559931,4.25485,1454.160086,1414.79221
146,6.3,2.5,5.0,1.9,2,0.375219,0.178483,0.446298,4.25485,1454.160086,1414.79221
147,6.5,3.0,5.2,2.0,2,0.240008,0.129790,0.630202,4.25485,1454.160086,1414.79221
148,6.2,3.4,5.4,2.3,2,0.316128,0.182531,0.501342,4.25485,1454.160086,1414.79221


In [423]:
df.to_csv('./iris_csvFile/Iris-m5-addF2.csv',index = False)

## FECBO-DB(M=8)へのF2の記入

In [424]:
df = pd.read_csv("./iris_csvFile/Iris-m8.csv")
u = df.loc[:,"u0":"u2"].values
X = df.loc[:,"sepal_length":"petal_width"].values
F = df["F"].values[0]
J = df["J"].values[0]
cluster = df["cluster"].values
m = 8
u_m = u**m
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2,J,F
0,5.1,3.5,1.4,0.2,1.0,0.219156,0.554037,0.226808,0.180122,1785.63066
1,4.9,3.0,1.4,0.2,1.0,0.246973,0.489700,0.263327,0.180122,1785.63066
2,4.7,3.2,1.3,0.2,1.0,0.246139,0.492158,0.261703,0.180122,1785.63066
3,4.6,3.1,1.5,0.2,1.0,0.253093,0.476265,0.270642,0.180122,1785.63066
4,5.0,3.6,1.4,0.2,1.0,0.228459,0.532209,0.239331,0.180122,1785.63066
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,0.454356,0.230701,0.314944,0.180122,1785.63066
146,6.3,2.5,5.0,1.9,0.0,0.404182,0.236896,0.358923,0.180122,1785.63066
147,6.5,3.0,5.2,2.0,0.0,0.531060,0.202124,0.266816,0.180122,1785.63066
148,6.2,3.4,5.4,2.3,0.0,0.425264,0.243249,0.331487,0.180122,1785.63066


In [425]:
v = v_calculation(u_m,c,dimension,X)
v

array([[6.53034898, 2.9960448 , 5.32542146, 1.93410999],
       [5.01617402, 3.38815083, 1.50354923, 0.2440358 ],
       [5.98336877, 2.84097677, 4.43556794, 1.42835395]])

In [429]:
cluster = cluster.astype(np.int64)
cluster.dtype

dtype('int64')

In [430]:
F_1,F_2 = calc_F(c,cluster,X,v,m,u)

center_distance
[[ 0.         19.90953107  1.37086191]
 [19.90953107  0.         11.23420833]
 [ 1.37086191 11.23420833  0.        ]]
d[0] [20.71594016  0.03219863 11.93817815]
new_d[0] = d_{ki}+d_{ii*} [4.06254712e+01 3.21986279e-02 2.31723865e+01]
Jの値: 0.18012164024442
Jの値を再計算: 0.18012164024441357
F_1の値: 1785.6306597921148
F_2の値: 1749.5919353479787


In [434]:
df["cluster"] = cluster
df["F2"] = F_2
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2,J,F,F2
0,5.1,3.5,1.4,0.2,1,0.219156,0.554037,0.226808,0.180122,1785.63066,1749.591935
1,4.9,3.0,1.4,0.2,1,0.246973,0.489700,0.263327,0.180122,1785.63066,1749.591935
2,4.7,3.2,1.3,0.2,1,0.246139,0.492158,0.261703,0.180122,1785.63066,1749.591935
3,4.6,3.1,1.5,0.2,1,0.253093,0.476265,0.270642,0.180122,1785.63066,1749.591935
4,5.0,3.6,1.4,0.2,1,0.228459,0.532209,0.239331,0.180122,1785.63066,1749.591935
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,0.454356,0.230701,0.314944,0.180122,1785.63066,1749.591935
146,6.3,2.5,5.0,1.9,0,0.404182,0.236896,0.358923,0.180122,1785.63066,1749.591935
147,6.5,3.0,5.2,2.0,0,0.531060,0.202124,0.266816,0.180122,1785.63066,1749.591935
148,6.2,3.4,5.4,2.3,0,0.425264,0.243249,0.331487,0.180122,1785.63066,1749.591935


In [436]:
df.to_csv(f'./iris_csvFile/Iris-m{m}-addF2.csv',index = False)

# FECBO-E系

In [471]:
#hcm、fecbo-e用クラスタ中心計算部分関数作成
def v_calculation(u,c,dimension,X):
    #分母計算
    v_denominator = np.sum(u,axis=0)
    #分子計算
    v_numerator = np.zeros((c,dimension))
    for j in range(c):
        for i in range(X_size):
            v_numerator[j]+= u[i][j]*X[i]
   
    v = np.zeros((c,dimension ))
    for i in range(c):
        v[i] = v_numerator[i]/v_denominator[i]
    return v

## FECBO-E(lambda=0.1)

In [462]:
df = pd.read_csv("./iris_csvFile/Iris-E0.1.csv")
u = df.loc[:,"u1":"u3"].values
X = df.loc[:,"sepal_length":"petal_width"].values
F = df["F"].values[0]
J = df["J"].values[0]
cluster = df["cluster"].values
#正則化パラメーター設定
lam = 0.1
lam_1= 1/lam
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F
0,5.1,3.5,1.4,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244
1,4.9,3.0,1.4,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244
2,4.7,3.2,1.3,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244
3,4.6,3.1,1.5,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244
4,5.0,3.6,1.4,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,0.333333,0.333333,0.333333,-967.094033,680.8244
146,6.3,2.5,5.0,1.9,2,0.333333,0.333333,0.333333,-967.094033,680.8244
147,6.5,3.0,5.2,2.0,2,0.333333,0.333333,0.333333,-967.094033,680.8244
148,6.2,3.4,5.4,2.3,2,0.333333,0.333333,0.333333,-967.094033,680.8244


In [465]:
v = v_calculation(u,c,dimension,X)
v

array([[5.84333314, 3.05400004, 3.75866622, 1.19866648],
       [5.8433334 , 3.05399998, 3.75866684, 1.19866674],
       [5.84333345, 3.05399997, 3.75866695, 1.19866678]])

In [466]:
F_1,F_2 = calc_F(c,cluster,X,v,lam_1,u,"fecbo-e")

center_distance
[[0.00000000e+00 5.22225014e-13 7.29843534e-13]
 [5.22225014e-13 0.00000000e+00 1.73336781e-14]
 [7.29843534e-13 1.73336781e-14 0.00000000e+00]]
d[0] [7.31210118 7.31210506 7.31210576]
new_d[0] = d_{ki}+d_{ii*} [7.31210118 7.31210506 7.31210576]
Jの値: -967.0940330021606
Jの値を再計算: -967.0940330021604
F_1の値: 680.8244000000261
F_2の値: 453.88295093748854


In [468]:
df["F2"] = F_2
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F,F2
0,5.1,3.5,1.4,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244,453.882951
1,4.9,3.0,1.4,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244,453.882951
2,4.7,3.2,1.3,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244,453.882951
3,4.6,3.1,1.5,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244,453.882951
4,5.0,3.6,1.4,0.2,0,0.333333,0.333333,0.333333,-967.094033,680.8244,453.882951
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,0.333333,0.333333,0.333333,-967.094033,680.8244,453.882951
146,6.3,2.5,5.0,1.9,2,0.333333,0.333333,0.333333,-967.094033,680.8244,453.882951
147,6.5,3.0,5.2,2.0,2,0.333333,0.333333,0.333333,-967.094033,680.8244,453.882951
148,6.2,3.4,5.4,2.3,2,0.333333,0.333333,0.333333,-967.094033,680.8244,453.882951


In [469]:
df.to_csv(f'./iris_csvFile/Iris-E0.1-addF2.csv',index = False)

## FECBO-E(lambda=0.5)

In [472]:
df = pd.read_csv("./iris_csvFile/Iris-E0.5.csv")
u = df.loc[:,"u1":"u3"].values
X = df.loc[:,"sepal_length":"petal_width"].values
F = df["F"].values[0]
J = df["J"].values[0]
cluster = df["cluster"].values
#正則化パラメーター設定
lam = 0.5
lam_1= 1/lam
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F
0,5.1,3.5,1.4,0.2,1,0.000930,0.999001,0.000069,10.716422,190.993843
1,4.9,3.0,1.4,0.2,1,0.000996,0.998941,0.000064,10.716422,190.993843
2,4.7,3.2,1.3,0.2,1,0.000525,0.999445,0.000030,10.716422,190.993843
3,4.6,3.1,1.5,0.2,1,0.000933,0.999011,0.000057,10.716422,190.993843
4,5.0,3.6,1.4,0.2,1,0.000790,0.999153,0.000057,10.716422,190.993843
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,0.385891,0.000025,0.614084,10.716422,190.993843
146,6.3,2.5,5.0,1.9,2,0.489348,0.000116,0.510536,10.716422,190.993843
147,6.5,3.0,5.2,2.0,2,0.421828,0.000050,0.578122,10.716422,190.993843
148,6.2,3.4,5.4,2.3,2,0.395205,0.000030,0.604766,10.716422,190.993843


In [473]:
v = v_calculation(u,c,dimension,X)
v

array([[6.08623282, 2.81662154, 4.66943467, 1.56942083],
       [5.01352883, 3.38092855, 1.53830182, 0.27598126],
       [6.47974663, 2.94578573, 5.19918759, 1.80505   ]])

In [476]:
F_1,F_2 = calc_F(c,cluster,X,v,lam_1,u,"fecbo-e")

center_distance
[[ 0.         12.94611507  0.50769577]
 [12.94611507  0.         18.0792797 ]
 [ 0.50769577 18.0792797   0.        ]]
d[0] [14.00417776  0.04655582 19.22086605]
new_d[0] = d_{ki}+d_{ii*} [26.95029283  0.04655582 37.30014574]
Jの値: 10.716422103322715
Jの値を再計算: 10.716422103321602
F_1の値: 190.99384251425911
F_2の値: 112.89702306218848


In [477]:
df["F2"] = F_2
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F,F2
0,5.1,3.5,1.4,0.2,1,0.000930,0.999001,0.000069,10.716422,190.993843,112.897023
1,4.9,3.0,1.4,0.2,1,0.000996,0.998941,0.000064,10.716422,190.993843,112.897023
2,4.7,3.2,1.3,0.2,1,0.000525,0.999445,0.000030,10.716422,190.993843,112.897023
3,4.6,3.1,1.5,0.2,1,0.000933,0.999011,0.000057,10.716422,190.993843,112.897023
4,5.0,3.6,1.4,0.2,1,0.000790,0.999153,0.000057,10.716422,190.993843,112.897023
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,0.385891,0.000025,0.614084,10.716422,190.993843,112.897023
146,6.3,2.5,5.0,1.9,2,0.489348,0.000116,0.510536,10.716422,190.993843,112.897023
147,6.5,3.0,5.2,2.0,2,0.421828,0.000050,0.578122,10.716422,190.993843,112.897023
148,6.2,3.4,5.4,2.3,2,0.395205,0.000030,0.604766,10.716422,190.993843,112.897023


In [478]:
df.to_csv(f'./iris_csvFile/Iris-E0.5-addF2.csv',index = False)

## FECBO-E(lambda=1)

In [479]:
df = pd.read_csv("./iris_csvFile/Iris-E1.csv")
u = df.loc[:,"u1":"u3"].values
X = df.loc[:,"sepal_length":"petal_width"].values
F = df["F"].values[0]
J = df["J"].values[0]
cluster = df["cluster"].values
#正則化パラメーター設定
lam = 1
lam_1= 1/lam
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F
0,5.1,3.5,1.4,0.2,2,1.305501e-10,0.000010,9.999896e-01,62.062101,137.167362
1,4.9,3.0,1.4,0.2,2,9.811976e-11,0.000014,9.999857e-01,62.062101,137.167362
2,4.7,3.2,1.3,0.2,2,1.905782e-11,0.000004,9.999957e-01,62.062101,137.167362
3,4.6,3.1,1.5,0.2,2,7.376430e-11,0.000013,9.999868e-01,62.062101,137.167362
4,5.0,3.6,1.4,0.2,2,8.611843e-11,0.000008,9.999924e-01,62.062101,137.167362
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,8.801610e-01,0.119839,7.459083e-10,62.062101,137.167362
146,6.3,2.5,5.0,1.9,0,5.422487e-01,0.457751,2.376527e-08,62.062101,137.167362
147,6.5,3.0,5.2,2.0,0,7.924845e-01,0.207516,3.691634e-09,62.062101,137.167362
148,6.2,3.4,5.4,2.3,0,8.645741e-01,0.135426,1.080897e-09,62.062101,137.167362


In [480]:
v = v_calculation(u,c,dimension,X)
v

array([[6.69631839, 3.02011852, 5.51024829, 1.96403267],
       [5.87769847, 2.74113853, 4.37201627, 1.41979773],
       [5.00649995, 3.41048162, 1.47700429, 0.25032203]])

In [483]:
F_1,F_2 = calc_F(c,cluster,X,v,lam_1,u,"fecbo-e")

center_distance
[[ 0.          2.33973219 22.21173102]
 [ 2.33973219  0.         10.95577484]
 [22.21173102 10.95577484  0.        ]]
d[0] [22.78447088 11.50147287  0.02521777]
new_d[0] = d_{ki}+d_{ii*} [4.49962019e+01 2.24572477e+01 2.52177658e-02]
Jの値: 62.0621010068738
Jの値を再計算: 62.062101006873355
F_1の値: 137.1673622003867
F_2の値: 65.39280535070297


In [484]:
df["F2"] = F_2
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F,F2
0,5.1,3.5,1.4,0.2,2,1.305501e-10,0.000010,9.999896e-01,62.062101,137.167362,65.392805
1,4.9,3.0,1.4,0.2,2,9.811976e-11,0.000014,9.999857e-01,62.062101,137.167362,65.392805
2,4.7,3.2,1.3,0.2,2,1.905782e-11,0.000004,9.999957e-01,62.062101,137.167362,65.392805
3,4.6,3.1,1.5,0.2,2,7.376430e-11,0.000013,9.999868e-01,62.062101,137.167362,65.392805
4,5.0,3.6,1.4,0.2,2,8.611843e-11,0.000008,9.999924e-01,62.062101,137.167362,65.392805
...,...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,8.801610e-01,0.119839,7.459083e-10,62.062101,137.167362,65.392805
146,6.3,2.5,5.0,1.9,0,5.422487e-01,0.457751,2.376527e-08,62.062101,137.167362,65.392805
147,6.5,3.0,5.2,2.0,0,7.924845e-01,0.207516,3.691634e-09,62.062101,137.167362,65.392805
148,6.2,3.4,5.4,2.3,0,8.645741e-01,0.135426,1.080897e-09,62.062101,137.167362,65.392805


In [486]:
df.to_csv('./iris_csvFile/Iris-E1-addF2.csv',index = False)

## FECBO-E(lambda=5)

In [543]:
df = pd.read_csv("./iris_csvFile/Iris-E5.csv")
u = df.loc[:,"u1":"u3"].values
X = df.loc[:,"sepal_length":"petal_width"].values
F = df["F"].values[0]
J = df["J"].values[0]
cluster = df["cluster"].values
#正則化パラメーター設定
lam = 5
lam_1= 1/lam
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u1,u2,u3,J,F
0,5.1,3.5,1.4,0.2,1,1.451213e-55,1.000000e+00,6.462483e-26,78.537011,85.427756
1,4.9,3.0,1.4,0.2,1,2.109315e-56,1.000000e+00,3.201050e-25,78.537011,85.427756
2,4.7,3.2,1.3,0.2,1,3.838633e-60,1.000000e+00,7.642006e-28,78.537011,85.427756
3,4.6,3.1,1.5,0.2,1,4.257260e-57,1.000000e+00,2.139303e-25,78.537011,85.427756
4,5.0,3.6,1.4,0.2,1,1.638034e-56,1.000000e+00,1.361329e-26,78.537011,85.427756
...,...,...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,9.998765e-01,4.469341e-46,1.235088e-04,78.537011,85.427756
146,6.3,2.5,5.0,1.9,2,1.657416e-01,1.404556e-37,8.342584e-01,78.537011,85.427756
147,6.5,3.0,5.2,2.0,0,9.945056e-01,4.060085e-42,5.494396e-03,78.537011,85.427756
148,6.2,3.4,5.4,2.3,0,9.997270e-01,3.514714e-45,2.729775e-04,78.537011,85.427756


In [545]:
u[0].dtype

dtype('float64')

In [492]:
v = v_calculation(u,c,dimension,X)
v

array([[6.83972701, 3.07019585, 5.71267715, 2.06145108],
       [5.006004  , 3.41796075, 1.46406568, 0.24403658],
       [5.88996246, 2.74436734, 4.38653588, 1.42777494]])

In [493]:
F_1,F_2 = calc_F(c,cluster,X,v,lam_1,u,"fecbo-e")

center_distance
[[ 0.         24.83717544  3.16841306]
 [24.83717544  0.         11.17717922]
 [ 3.16841306 11.17717922  0.        ]]
d[0] [2.52755661e+01 2.16093175e-02 1.16218493e+01]
new_d[0] = d_{ki}+d_{ii*} [5.01127415e+01 2.16093175e-02 2.27990285e+01]
Jの値: 78.53701055268826
Jの値を再計算: 78.53701055268812
F_1の値: 85.42775624108042
F_2の値: 8.52856415652147


In [494]:
df.to_csv('./iris_csvFile/Iris-E5-addF2.csv',index = False)

## FECBO-E(lambda=100)

In [523]:
df = pd.read_csv("./iris_csvFile/Iris-E100.csv")
F = df.loc[151,"sepal_width"]
df = df.loc[:149,:]
u = df.loc[:,"u0":"u2"].values
X = df.loc[:,"sepal_length":"petal_width"].values
cluster = df["cluster"].values
cluster = cluster.astype(np.int64)
#正則化パラメーター設定
lam = 100
lam_1= 1/lam
X_size,dimension = X.shape
df

,sepal_length,sepal_width,petal_length,petal_width,cluster,u0,u1,u2
0,5.1,3.5,1.4,0.2,2.0,-2.187419e-10,-1.551752e-10,1.000000e+00
1,4.9,3.0,1.4,0.2,2.0,-2.191485e-10,-1.501515e-10,1.000000e+00
2,4.7,3.2,1.3,0.2,2.0,-2.213021e-10,-1.648261e-10,1.000000e+00
3,4.6,3.1,1.5,0.2,2.0,-2.195505e-10,-1.511337e-10,1.000000e+00
4,5.0,3.6,1.4,0.2,2.0,-2.193717e-10,-1.590264e-10,1.000000e+00
...,...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0.0,1.000000e+00,2.323658e-10,-2.187839e-10
146,6.3,2.5,5.0,1.9,0.0,1.000000e+00,2.632210e-09,-2.095636e-10
147,6.5,3.0,5.2,2.0,0.0,1.000000e+00,4.507358e-10,-2.154720e-10
148,6.2,3.4,5.4,2.3,0.0,1.000000e+00,2.610254e-10,-2.181276e-10


In [524]:
v = v_calculation(u,c,dimension,X)
v

TypeError: can't multiply sequence by non-int of type 'float'

In [549]:
X[0][0].dtype

dtype('float64')

In [542]:
X[0].dtype

dtype('float64')

In [540]:
u[0].dtype

dtype('float64')

In [551]:
u[0][0].dtype

dtype('float64')